In [1]:
# imports
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from numpy.random import normal
import pandas as pd

In [2]:
# set solver
opt = pyo.SolverFactory('cplex')

In [3]:
# define master model
master_model = pyo.ConcreteModel()

master_model.y = pyo.Var([1, 2], domain=pyo.PositiveReals)
master_model.z = pyo.Var([1, 2])

master_model.objecitve = pyo.Objective(expr = 100 * master_model.y[1] + 150 * master_model.y[2], sense=pyo.minimize)

master_model.constrains_1 = pyo.Constraint(expr = master_model.y[1] + master_model.y[2] <= 120)
master_model.constraint_2 = pyo.Constraint(expr = master_model.y[1] >= 40)
master_model.constraint_3 = pyo.Constraint(expr = master_model.y[2] >= 20)
master_model.cuts = pyo.ConstraintList()

In [4]:
# define sub-problems
def sub_problem_generator(master_model, objective_func_coef, first_stage_coef, sec_stage_coef, rhs):
    model_sp = pyo.ConcreteModel()    
    model_sp.x = pyo.Var([1, 2], domain=pyo.PositiveReals)    
    model_sp.objective = pyo.Objective(expr = objective_func_coef[0] * model_sp.x[1] + objective_func_coef[1] *
                                       model_sp.x[2], sense=pyo.minimize)    
    model_sp.constraint_1 = pyo.Constraint(expr = first_stage_coef[0] * master_model.y[1].value +
                                           sec_stage_coef[0][0] * model_sp.x[1] +
                                           sec_stage_coef[0][1] * model_sp.x[2] <= rhs[0])    
    model_sp.constraint_2 = pyo.Constraint(expr = first_stage_coef[1] * master_model.y[2].value +
                                           sec_stage_coef[1][0] * model_sp.x[1] +
                                           sec_stage_coef[1][1] * model_sp.x[2] <= rhs[1])    
    model_sp.constraint_3 = pyo.Constraint(expr = sec_stage_coef[2][0] * model_sp.x[1] <= rhs[2])    
    model_sp.constraint_4 = pyo.Constraint(expr = sec_stage_coef[3][1] * model_sp.x[2] <= rhs[3])    
    return model_sp

In [5]:
# return the dual variables
def dual_variables(model):
    duals = []
    for constraint in model.component_objects(pyo.Constraint, active=True):
        duals.append(model_sp_1.dual[constraint])
    return duals 

In [6]:
# define blocks coefficients
first_stage_coeff = [-60, -80]
second_stage_coeff = [[6, 10],[8, 5],[1, 0],[0, 1]]
rhs = [[0, 0, 500, 100], [0, 0, 300, 300]]

In [7]:
# solve master problem
master_solution = opt.solve(master_model)
master_model.display()

Model unknown

  Variables:
    y : Size=2, Index=y_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  40.0 :  None : False : False : PositiveReals
          2 :     0 :  20.0 :  None : False : False : PositiveReals
    z : Size=2, Index=z_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals

  Objectives:
    objecitve : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 7000.0

  Constraints:
    constrains_1 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 60.0 : 120.0
    constraint_2 : Size=1
        Key  : Lower : Body : Upper
        None :  40.0 : 40.0 :  None
    constraint_3 : Size=1
        Key  : Lower : Body : Upper
        None :  20.0 : 20.0 :  None
    cuts : Size=0
        Key : Lower : Body : Upper


In [8]:
# solve first sub-problem
model_sp_1 = sub_problem_generator(master_model, [-24, -28], first_stage_coeff, second_stage_coeff, rhs[0])
model_sp_1.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

sp_1_solution = opt.solve(model_sp_1)
model_sp_1.display()
print("******************")
model_sp_1.dual.display()

                
# retrive the dual variables
duals = dual_variables(model_sp_1)

Model unknown

  Variables:
    x : Size=2, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 : 137.5 :  None : False : False : PositiveReals
          2 :     0 : 100.0 :  None : False : False : PositiveReals

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : -6100.0

  Constraints:
    constraint_1 : Size=1
        Key  : Lower : Body   : Upper
        None :  None : -575.0 :   0.0
    constraint_2 : Size=1
        Key  : Lower : Body : Upper
        None :  None :  0.0 :   0.0
    constraint_3 : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 137.5 : 500.0
    constraint_4 : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 100.0 : 100.0
******************
dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key          : Value
    constraint_1 :   0.0
    constraint_2 :  -3.0
    constraint_3 :   0.0
    constraint_4 : -13.0


In [9]:
# add the cut
master_model.cuts.add((-first_stage_coeff[0]) * master_model.y[1] * duals[0] - first_stage_coeff[1] * master_model.y[2]
                      * duals[1]  + rhs[0][2] * duals[2] + rhs[0][3] * duals[3] <= master_model.z[1])

In [10]:
# solve second sub-problem
model_sp_2 = sub_problem_generator(master_model, [-28, -32], first_stage_coeff, second_stage_coeff, rhs[1])
model_sp_2.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# retrive the dual variables
duals = dual_variables(model_sp_1)

# add the cut
master_model.cuts.add((-first_stage_coeff[0]) * master_model.y[1] * duals[0] - first_stage_coeff[1] * master_model.y[2] 
                      * duals[1]  + rhs[1][2] * duals[2] + rhs[1][3] * duals[3] <= master_model.z[2])

In [11]:
# resolve the master model
master_model.objecitve.expr += 0.4 * master_model.z[1] + 0.6 * master_model.z[2]
master_solution = opt.solve(master_model)
master_model.display()

Model unknown

  Variables:
    y : Size=2, Index=y_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  40.0 :  None : False : False : PositiveReals
          2 :     0 :  80.0 :  None : False : False : PositiveReals
    z : Size=2, Index=z_index
        Key : Lower : Value    : Upper : Fixed : Stale : Domain
          1 :  None : -20500.0 :  None : False : False :  Reals
          2 :  None : -23100.0 :  None : False : False :  Reals

  Objectives:
    objecitve : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : -6060.0

  Constraints:
    constrains_1 : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 120.0 : 120.0
    constraint_2 : Size=1
        Key  : Lower : Body : Upper
        None :  40.0 : 40.0 :  None
    constraint_3 : Size=1
        Key  : Lower : Body : Upper
        None :  20.0 : 80.0 :  None
    cuts : Size=2
        Key : Lower : Body : Upper
          1 :  None :  0.0 :   0.